In [2]:
# Built-in library
import itertools
import re
from typing import Any, Optional, Sequence, Union
import warnings

# Standard imports
import numpy as np
import pandas as pd
import random
import string
from datetime import datetime, timedelta

warnings.filterwarnings("ignore")


# pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 2_000

# Black code formatter (Optional)
%load_ext nb_black
# auto reload imports
%load_ext autoreload
%autoreload 2

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

# CREATING ORDER AND MERGING DATA VERSION TWO

In [6]:
class OrderRestaurant:
    """This class create a mock data for dish order using some information of related data
    It accept one argument which is the number of order data you want
    """

    def __init__(self, *, num_order: int) -> None:
        self.num_order = num_order
        self.dict_ = {
            "customer_id": [],
            "product_id": [],
            "distance": [],
            "product_rating": [],
            "date": [],
        }
        # self.len_customer_data = 956
        self.list_customers_id = range(0, 957)
        self.list_drink_id = range(0, 1230)
        self.list_meal_id = range(1230, 2444)
        self.list_snacks_id = range(2444, 3735)
        self.list_restaurant_snacks_id = range(2444, 3368)
        self.list_cafes_snacks_id = range(3368, 3735)
        self.list_restaurant_drink_id = range(0, 863)
        self.list_cafes_drink_id = range(863, 1229)
        
        # NOTE ALL THE ABOVE INFORMATION ARE BASED ON THE CUSTOMER, MENU, AND PLACE(resaturant/cafes) DATA
        # IF THE DATA CHANGES THIS VALUES WILL ALSO CHANGE TOO

    def append_row(self, *, list_product_id: list):
        """
        This function add a row to a dictionary that will be story the data

        param:
           list_product_id = is a list of the product id that would be use to randomly select the dish
        """
        # generate customer id
        customer_id = random.choice(self.list_customers_id)
        self.dict_["customer_id"].append(customer_id)  # update customer id

        # generate product id
        product_id = random.choice(list_product_id)
        self.dict_["product_id"].append(product_id)  # update meal id

        # generate meal rating
        values = [1, 2, 3, 4, 5]
        weights = [0.1, 0.2, 0.2, 0.1, 0.4]
        meal_rating = random.choices(values, weights=weights)[0]
        self.dict_["product_rating"].append(meal_rating)  # update meal rating

        # generate distance
        weights = [0.3, 0.5, 0.2]
        ranges = [(0.9, 12), (13, 30), (31, 47)]
        # Generate a random value with weighted ranges
        distance = random.choices(
            population=[round(random.uniform(start, end), 2) for start, end in ranges],
            weights=weights,
            k=1,
        )[0]
        self.dict_["distance"].append(distance)

        # generate year
        year = 22
        month = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        month_weight = [1.3, 1, 0.3, 0.8, 1.1, 0.5, 1.2, 0.5, 0.4, 0.5, 1, 1.4]

        month_with_thirty_days = [4, 6, 9, 11]
        mth = random.choices(month, weights=month_weight)[0]
        if mth not in month_with_thirty_days and mth != 2:
            day = random.randint(1, 31)
        elif mth == 2:
            day = random.randint(1, 28)
        else:
            day = random.randint(1, 30)

        time = f"{day}-{mth}-{year}"
        self.dict_["date"].append(time)

        return self.dict_

    def generate_order_data(self) -> pd.DataFrame:
        """This function create a each row by randomly selecting the values based on some specifications
        and then return the result in a datarame format
        """
        # loop through depending on the number on oder that has been specified
        for i in range(self.num_order):
            product_type = ["meal", "snacks", "drink"]
            product_weight = [0.35782, 0.38236, 0.25982]
            get_product = random.choices(product_type, weights=product_weight)[0]

            if get_product == "meal":
                self.append_row(list_product_id=self.list_meal_id)

            elif get_product == "snacks":
                place = ["restaurant", "cafes"]
                place_weight = [0.5, 0.6]
                get_place = random.choices(place, weights=place_weight)[0]

                if get_place == "restaurant":
                    self.append_row(list_product_id=self.list_restaurant_snacks_id)

                else:
                    get_place == "cafes"
                    self.append_row(list_product_id=self.list_cafes_snacks_id)

            else:
                place = ["restaurant", "cafes"]
                place_weight = [0.3, 0.7]
                get_place = random.choices(place, weights=place_weight)[0]

                if get_place == "restaurant":
                    self.append_row(list_product_id=self.list_restaurant_drink_id)

                else:
                    get_place == "cafes"
                    self.append_row(list_product_id=self.list_cafes_drink_id)

        # create a dataframe for the dict
        df = pd.DataFrame(self.dict_)

        # set order id
        df = df.reset_index().rename(columns={"index": "order_id"})

        # adding a review to the dataframe
        condition = [
            (df["product_rating"] >= 4),
            (df["product_rating"] < 3),
            (df["product_rating"] == 3),
        ]
        choice = [1, -1, 0]
        df["review"] = np.select(condlist=condition, choicelist=choice)

        # generating the time funtion
        def generate_time(*, val: float) -> int:
            if val <= 5:
                time = random.randint(10, 25)
            elif val <= 10:
                time = random.randint(13, 30)
            elif val <= 15:
                time = random.randint(19, 35)
            elif val <= 20:
                time = random.randint(22, 40)
            elif val <= 25:
                time = random.randint(27, 45)
            elif val <= 30:
                time = random.randint(30, 55)
            elif val <= 35:
                time = random.randint(34, 59)
            elif val <= 40:
                time = random.randint(39, 64)
            else:
                time = random.randint(42, 70)
            return time

        # apply the function
        df["time_taken"] = df["distance"].apply(lambda x: generate_time(val=x))

        return df

<IPython.core.display.Javascript object>

In [9]:
df2.head(3)

,order_id,customer_id,product_id,distance,product_rating,date,review,time_taken
0,0,363,2411,3.08,5,24-11-22,1,16
1,1,593,3535,9.42,5,11-11-22,1,24
2,2,489,1131,4.97,5,3-6-22,1,19


<IPython.core.display.Javascript object>

In [10]:
df2.shape

(9024, 8)

<IPython.core.display.Javascript object>

## MERGING THE DATA

In [12]:
# order data
order = pd.read_csv("data/order_V2.csv")

# customers data
customers = pd.read_csv("data/customers.csv")

# restaurants data
place = pd.read_csv("data/restaurant_cafes.csv")

# meal data
product = pd.read_csv("data/menu.csv")

<IPython.core.display.Javascript object>

In [13]:
place.head(3)

,place_id,place_type,place_name,place_address,place_phone_no
0,0,restaurant,Bernadines Cloud Kitchen,"35, Sholanke Akoka, Yaba, Lagos",0903 651 7676
1,1,restaurant,Abibiz Restaurant,"Murtala Mohammed International Airport, Ikeja, Lagos, Nigeria","0803 302 4280, 01 773 1431"
2,2,restaurant,Afi's Restaurant,"5 Olufemi Street, Surulere, Lagos, Nigeria",01 470 3044


<IPython.core.display.Javascript object>

In [14]:
product.head(3)

,product_id,place_id,product,product_description,product_type
0,0,0,Soya Milk,"Soya Milk is a refreshing nutritious drink made from plant protein. It made from soya beans, and is also very delicious. Soy milk is rich in sodium, potassium, calcium and iron.",drink
1,1,0,fura da nono,It is made from fermented milk and ground millet grains and is thick but not as thick as yoghurt.,drink
2,2,0,Kunun zaki,"It is made by first allowing the grain seeds to germinate, and then the seeds are soaked in water for a few days after which it is blended together with sweet potatoes and ginger or pepper to form a smooth paste.",drink


<IPython.core.display.Javascript object>

In [15]:
customers.head(3)

,customer_id,customer_name,customer_address,customer_phone_num,customer_email_address
0,0,Alan Perry,DU10 Ikeja Along Road Lagos,497-733-4177,AlanPerry@yahoo.com
1,1,Robin Henry,PM2 Maryland Mall Road Lagos,632-957-7882,RobinHenry@gmail.com
2,2,Kelly Hoffman,CV51 Marina Street Lagos,115-847-9886,KellyHoffman@yahoo.com


<IPython.core.display.Javascript object>

In [16]:
order.head()

,order_id,customer_id,product_id,distance,product_rating,date,review,time_taken
0,0,0,3711,46.99,3,6-4-22,0,43
1,1,665,3206,28.87,4,30-10-22,1,44
2,2,315,2717,44.05,5,27-6-22,1,55
3,3,888,3624,10.58,3,15-6-22,0,35
4,4,332,2315,3.88,3,6-10-22,0,10


<IPython.core.display.Javascript object>

In [17]:
# merging the data
df_combined = df2.merge(product, on="product_id", how="left").merge(
    place, on="place_id", how="left"
)
df_combined

,order_id,customer_id,product_id,distance,product_rating,date,review,time_taken,place_id,product,product_description,product_type,place_type,place_name,place_address,place_phone_no
0,0,363,2411,3.08,5,24-11-22,1,16,67,Moin/Moi Moi,"A traditional food made from beans, peppers, spices, and seasonings. This inexpensive and nutritious meal can be served any time of the day. Asides from the special note it adds as a side dish, it is common street food in Nigeria that s loved by adults and kids.",meal,restaurant,Topever Restaurant,"39, Market Street, Shomolu, Shomolu, Lagos, Nigeria","0806 209 8455, 0803 488 6706"
1,1,593,3535,9.42,5,11-11-22,1,24,78,Meat Pie,"A mix of flour, butter, salt, sugar, baking powder, and egg batter kneaded and filled with seasoned ground beef, carrots and potatoes make a comfort snack for you. Whether you are eating it as brunch, breakfast, or snack, you will enjoy this light and flaky delicacy.",snacks,cafes,Ziya Delicacy Boutique,"26B Oju Olobun Cl, Lagos, Nigeria",NaN
2,2,489,1131,4.97,5,3-6-22,1,19,84,Zobo,"Zobo is made by boiling the hibiscus plant flower in water together with other ingredients such as ginger, garlic, pineapple and the rest of them you wish to add to your own taste.",drink,cafes,Cafe One Yaba,"E-Center, 1-11 Commercial Ave, Sabo, Yaba, Lagos, Nigeria",NaN
3,3,901,2376,33.60,3,12-5-22,0,57,65,Ewedu – Jute Leaves Soup,"This is a unique soup made from jute leaves. An easy-make soup with a slimy texture to aid the eating of swallows like eba, amala and semo. This green soup is rich and delicious. You can serve it with beans soup, beef, fish, or chicken stew.",meal,restaurant,Sweetspot Eatery And African Dishes,"20, Oladoyinbo Street, Ikeja, Lagos, Nigeria","0703 384 2618, 0807 595 3779"
4,4,822,1979,21.44,1,7-9-22,-1,43,42,Yam Pottage/Yam Porridge,"This yam recipe uses puna yam boiled and mashed with pepper mix blend, onions, seasoning, and palm oil. It has a bright color with an amazing taste. Garnish it with any leafy vegetable and serve it with the protein of your choice.",meal,restaurant,Karma Restaurant,"17, Burma Road, Apapa, Lagos, Nigeria","0816 569 6021, 0909 034 9081"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9019,9019,607,83,9.61,5,10-5-22,1,27,6,Pineapple Drink,"Made with the normally discarded bits of pineapple – the skin and core, this has a distinct, clear pineapple flavour. Spice with cloves, ginger and other aromatics.",drink,restaurant,Appetizers Restaurant,"66 Ijaiye Road, Suite 33, Assembly Spot Complex, Ikeja, Lagos, Nigeria",0803 727 7483
9020,9020,188,1597,18.10,4,13-9-22,1,32,20,Jollof Spaghetti,"This is not a regular everyday spaghetti that you eat with ready-made stew. It is a one-pot stewed spaghetti recipe made with aromatics, vegetable oil, and pepper. You can stir your choice of protein and eat it for lunch or dinner.",meal,restaurant,China Town Express Restaurant,"14 Idowu Martins Street, Lagos Island, Lagos, Nigeria","0704 095 4224, 0806 454 2582"
9021,9021,72,1844,23.74,5,23-9-22,1,28,34,Pounded Yam,"We call this the king of swallows. It is made from African yam pounded till perfectly smooth and served like a dough. Pounded yam is paired with African soups like egusi, okro soup, and stewed spinach.",meal,restaurant,Enjoyurmouth.com,"Lekki Ajah, Lagos, Lagos State Nigeria",088034310244
9022,9022,169,937,46.17,3,25-1-22,0,60,73,Soya Milk,"Soya Milk is a refreshing nutritious drink made from plant protein. It made from soya beans, and is also very delicious. Soy milk is rich in sodium, potassium, calcium and iron.",drink,cafes,MyYa’s Cafe,"Plot 1, Block, 4 Admiralty Rd, Lekki Phase I, Lagos, Nigeria",NaN


<IPython.core.display.Javascript object>

In [18]:
df_combined.shape

(9024, 16)

<IPython.core.display.Javascript object>

### Below this notebook is the version one of this work

#### and the difference  between them is that version to consider the percentage of product to be found in restaurant and cafes while version one does'nt consider that

# CREATING ORDER AND MERGING DATA VERSION ONE

In [193]:
class Order:
    def __init__(self, *, num_order: int) -> None:
        self.num_order = num_order
        self.dict_ = {
            "customer_id": [],
            "place_id": [],
            "product_id": [],
            "distance": [],
            "product_rating": [],
            "date": [],
        }
        self.len_customer_data = 10100
        self.list_meal_id = range(0, 48)
        self.list_drink_id = range(48, 70)
        self.list_snacks_id = range(70, 91)
        self.list_customers_id = range(self.len_customer_data)
        self.list_restaurant_id = range(0, 69)
        self.list_cafes_id = range(69, 90)

    def append_row(self, *, list_place_id: list, list_product_id: list):
        # generate customer id
        customer_id = random.choice(self.list_customers_id)
        self.dict_["customer_id"].append(customer_id)  # update customer id

        # generate place id
        place_id = random.choice(list_place_id)
        self.dict_["place_id"].append(place_id)  # update restaurant/cafe id

        # generate product id
        product_id = random.choice(list_product_id)
        self.dict_["product_id"].append(product_id)  # update meal id

        # generate meal rating
        values = [1, 2, 3, 4, 5]
        weights = [0.1, 0.2, 0.2, 0.1, 0.4]
        meal_rating = random.choices(values, weights=weights)[0]
        self.dict_["product_rating"].append(meal_rating)  # update meal rating

        # generate distance
        weights = [0.3, 0.5, 0.2]
        ranges = [(0.9, 12), (13, 30), (31, 47)]

        # Generate a random value with weighted ranges
        distance = random.choices(
            population=[round(random.uniform(start, end), 2) for start, end in ranges],
            weights=weights,
            k=1,
        )[0]

        self.dict_["distance"].append(distance)

        # generate year
        year = 22
        month = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        month_weight = [1.3, 1, 0.3, 0.8, 1.1, 0.5, 1.2, 0.5, 0.4, 0.5, 1, 1.4]

        month_with_thirty_days = [4, 6, 9, 11]
        mth = random.choices(month, weights=month_weight)[0]
        if mth not in month_with_thirty_days and mth != 2:
            day = random.randint(1, 31)
        elif mth == 2:
            day = random.randint(1, 28)
        else:
            day = random.randint(1, 30)

        time = f"{day}-{mth}-{year}"
        self.dict_["date"].append(time)

        return self.dict_

    def generate_order_data(self) -> pd.DataFrame:
        for i in range(self.num_order):
            place_type = ["restaurant", "cafes"]
            place_weight = [0.6, 0.4]
            get_place = random.choices(place_type, weights=place_weight)[0]

            if get_place == "restaurant":
                product_type = ["meal", "snacks", "drink"]
                product_weight = [0.6, 0.3, 0.1]
                get_product = random.choices(product_type, weights=product_weight)[0]

                if get_product == "meal":
                    self.append_row(
                        list_place_id=self.list_restaurant_id,
                        list_product_id=self.list_meal_id,
                    )

                elif get_product == "snacks":
                    self.append_row(
                        list_place_id=self.list_restaurant_id,
                        list_product_id=self.list_snacks_id,
                    )

                elif get_product == "drink":
                    self.append_row(
                        list_place_id=self.list_restaurant_id,
                        list_product_id=self.list_drink_id,
                    )

            elif get_place == "cafes":
                product_type = ["snacks", "drink"]
                product_weight = [0.5, 0.5]
                get_product = random.choices(product_type, weights=product_weight)[0]

                if get_product == "snacks":
                    self.append_row(
                        list_place_id=self.list_cafes_id,
                        list_product_id=self.list_snacks_id,
                    )

                elif get_product == "drink":
                    self.append_row(
                        list_place_id=self.list_cafes_id,
                        list_product_id=self.list_drink_id,
                    )

        # create a dataframe for the dict
        df = pd.DataFrame(self.dict_)

        # set order id
        df = df.reset_index().rename(columns={"index": "order_id"})

        # adding a review to the dataframe
        condition = [
            (df["product_rating"] >= 4),
            (df["product_rating"] < 3),
            (df["product_rating"] == 3),
        ]
        choice = [1, -1, 0]
        df["review"] = np.select(condlist=condition, choicelist=choice)

        # generating the time funtion
        def generate_time(*, val: float) -> int:
            if val <= 5:
                time = random.randint(10, 25)
            elif val <= 10:
                time = random.randint(13, 30)
            elif val <= 15:
                time = random.randint(19, 35)
            elif val <= 20:
                time = random.randint(22, 40)
            elif val <= 25:
                time = random.randint(27, 45)
            elif val <= 30:
                time = random.randint(30, 55)
            elif val <= 35:
                time = random.randint(34, 59)
            elif val <= 40:
                time = random.randint(39, 64)
            else:
                time = random.randint(42, 70)
            return time

        # apply the function
        df["time_taken"] = df["distance"].apply(lambda x: generate_time(val=x))

        return df

<IPython.core.display.Javascript object>

In [6]:
df

,order_id,customer_id,place_id,product_id,distance,product_rating,date,review,time_taken
0,0,250,84,60,3.13,5,26-1-22,1,11
1,1,146,79,90,16.87,5,7-9-22,1,25
2,2,492,82,79,7.53,4,30-9-22,1,30
3,3,424,23,15,33.66,5,4-7-22,1,43
4,4,291,85,62,29.97,4,29-1-22,1,39
...,...,...,...,...,...,...,...,...,...
12097,12097,37,42,72,23.99,4,2-8-22,1,30
12098,12098,326,9,20,15.76,3,11-1-22,0,37
12099,12099,492,70,82,39.69,4,27-1-22,1,42
12100,12100,423,66,12,5.71,3,8-6-22,0,25


<IPython.core.display.Javascript object>

## MERGING THE DATA

In [7]:
# order data
order = pd.read_csv("data/order.csv")

# customers data
customers = pd.read_csv("data/customers.csv")

# restaurants data
place = pd.read_csv("data/restaurant_cafes.csv")

# meal data
product = pd.read_csv("data/product.csv")

<IPython.core.display.Javascript object>

In [9]:
order.head(3)

,order_id,customer_id,place_id,product_id,distance,product_rating,date,review,time_taken
0,0,1540,74,84,11.23,1,18-7-22,-1,19
1,1,912,9,27,24.06,5,17-8-22,1,30
2,2,10006,32,70,8.40,2,10-8-22,-1,24


<IPython.core.display.Javascript object>

In [10]:
customers.head(3)

,customer_id,customer_name,customer_address,customer_phone_num,customer_email_address
0,0,Erika Garza,VY2 Lekki-Epe Expressway Lagos,997-815-9240,ErikaGarza@gmail.com
1,1,Nicole Guerra,OS78 Oba Akran Avenue Lagos,926-625-5359,NicoleGuerra@yahoo.com
2,2,Kari Moore,HB70 Oshodi-Apapa Expressway Lagos,463-204-7996,KariMoore@gmail.com


<IPython.core.display.Javascript object>

In [11]:
product.head(3)

,product_id,product,product_description,product_type
0,0,Jollof Rice,"The popular jollof rice is a classic recipe that makes it to almost all special occasions in Nigeria. It is a rich, tasty, and elegant meal consisting of rice cooked with pepper, broth, seasoning, and spices in one pot. Serve this amazing meal with any protein of your choice.",meal
1,1,Nigerian Fried Rice,"This version of fried rice consists of fresh vegetables fried and stirred into the rice. It is a healthy bowl that consists of beef liver, carrots, green beans, sweet corn peas, and more. You can add any protein of choice, like shrimp, chicken, or beef. Simple but insanely delicious.",meal
2,2,Egusi Soup,The list of Nigerian soups cannot be complete without Egusi soup. Egusi soup is made from melon seed from the gourd family. The melon seed is grounded and made into a ball with a mix of leafy vegetables and pepper. There are various ways of making egusi; you will like the Egusi Ijebu Recipe and Lumpy Egusi Stew. Served it with any swallow or rice to enjoy.,meal


<IPython.core.display.Javascript object>

In [16]:
place.head(3)

,place_id,place_type,place_name,place_address,place_phone_no
0,0,restaurant,Bernadines Cloud Kitchen,"35, Sholanke Akoka, Yaba, Lagos",0903 651 7676
1,1,restaurant,Abibiz Restaurant,"Murtala Mohammed International Airport, Ikeja, Lagos, Nigeria","0803 302 4280, 01 773 1431"
2,2,restaurant,Afi's Restaurant,"5 Olufemi Street, Surulere, Lagos, Nigeria",01 470 3044


<IPython.core.display.Javascript object>

In [8]:
# merging the data
df = df.merge(place, on="place_id", how="left").merge(
    product, on="product_id", how="left"
)
df

,order_id,customer_id,place_id,product_id,distance,product_rating,date,review,time_taken,place_type,place_name,place_address,place_phone_no,product,product_description,product_type
0,0,250,84,60,3.13,5,26-1-22,1,11,cafes,Cafe One Yaba,"E-Center, 1-11 Commercial Ave, Sabo, Yaba, Lagos, Nigeria",NaN,Filter Coffee,This is one of the earliest types of coffee; probably the first type of coffee served in Nigeria. Ground coffee beans is poured inside hot or cold water and left to steep. This is then filtered out and the resulting coffee is taken black or with milk.,drink
1,1,146,79,90,16.87,5,7-9-22,1,25,cafes,Bourbon House Cafe,"741 Adeola Hopewell St, Victoria Island, Lagos, Nigeria",NaN,Sandwich,"A sandwich is a food typically consisting of vegetables, sliced cheese or meat, placed on or between slices of bread, or more generally any dish wherein bread serves as a container or wrapper for another food type.",snacks
2,2,492,82,79,7.53,4,30-9-22,1,30,cafes,Grey Matter Social Space,"21C Akin Ogunlewe St, Victoria Island, Lagos, Nigeria",NaN,Pound Cake,"African pound cake is a good choice for you. In addition, this pound cake is chewy and requires pantry-staple ingredients. You easily bake this for snacks for school children or family picnics.",snacks
3,3,424,23,15,33.66,5,4-7-22,1,43,restaurant,Chow Mein Chinese Restaurant,"20, Prince Adelowo Adedeji Street, Lekki Estate Phase 1, Lagos, Nigeria",0818 734 3782,Moin/Moi Moi,"A traditional food made from beans, peppers, spices, and seasonings. This inexpensive and nutritious meal can be served any time of the day. Asides from the special note it adds as a side dish, it is common street food in Nigeria that s loved by adults and kids.",meal
4,4,291,85,62,29.97,4,29-1-22,1,39,cafes,Cafe One Yaba,"E-Center, 1-11 Commercial Ave, Sabo, Yaba, Lagos, Nigeria",NaN,Mojito,"Traditionally a Cuba drink, the mojito is a hugely popular cocktail and combines a mix of white rum, sugar (traditionally sugar cane juice), lime juice, sparkling water, and mint leaves",drink
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12097,12097,37,42,72,23.99,4,2-8-22,1,30,restaurant,Karma Restaurant,"17, Burma Road, Apapa, Lagos, Nigeria","0816 569 6021, 0909 034 9081",Chicken Pie,This is another version of meat pie. The difference from the classic one is that the filling is chicken instead of beef. It is loaded with chicken flavor and equally delicious.,snacks
12098,12098,326,9,20,15.76,3,11-1-22,0,37,restaurant,Bangkok Restaurant,"Plot 244A, Muri Okunntola Street, Victoria Island, Lagos, Nigeria","0803 307 7666, 0706 217 3888",Fufu,"Fufu is known throughout West Africa, and it tops the list on the menu of Nigerian swallows. Usually, the cassava is processed and cooked into a smooth, starchy, and stretchy dough. We don’t eat it alone. It is served with delicious soups like ewedu and egusi to accompany it down the throat.",meal
12099,12099,492,70,82,39.69,4,27-1-22,1,42,cafes,Pause | Café & Espresso Bar,"1 Bisway St, Lekki, Lagos, Nigeria",NaN,Doughnuts,"Doughnuts, with their delightful round shape and irresistible glazes, have captured the hearts of Nigerians with their sweet allure. These deep-fried treats are made from a yeast-based dough, typically flavored with nutmeg and often coated in a sugary glaze or sprinkled with powdered sugar.",snacks
12100,12100,423,66,12,5.71,3,8-6-22,0,25,restaurant,"The Kitchen, Lekki","21, Oladimeji Alo Street, Off Freedom Way, Lekki Phase 1, Lagos",0906 999 9996,Nigerian Egg Rolls,This filling snack features a boiled egg wrapped in a mass of flour dough and fried to form a ball-like shape. It is one of the street foods in Nigeria that gives you the satisfaction you want. Serve with cold juice at brunch or tea for breakfast.,meal


<IPython.core.display.Javascript object>